In [122]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize,CenterCrop,ColorJitter

In [115]:
pwd

'/opt/ml/code'

In [123]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '../input/data/eval'

## 1. Model 정의

In [68]:
'''class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
        
'''
torch.load


<function torch.serialization.load(f, map_location=None, pickle_module=<module 'pickle' from '/opt/conda/lib/python3.8/pickle.py'>, **pickle_load_args)>

## 2. Test Dataset 정의

In [124]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [128]:
# meta 데이터와 이미지 경로를 불러옵니다.

submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    CenterCrop((370,320)),
    Resize((272,232), Image.BILINEAR),
   # ColorJitter(0.2, 0.2, 0.2, 0.2),
    ToTensor(),
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False,
    num_workers=4,
    batch_size=32,
    #pin_memory=use_cuda,
    drop_last=False,

)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
#model = NfnetModel().to(device=device)
#model.load_state_dict(copyStateDict(torch.load('../models/best.pth')["state_dict"]))
#model.eval()
model = torch.load( 'recode/model/exp23/multi_03_009_90.80%_0.83.pt')
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_eff_exp23_03_009_3_nocolor.csv'), index=False)
print('test inference is done!')

test inference is done!


In [53]:
torch.load('../models/best.pth').keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'fc.weight', 'fc.bias'])

In [64]:
%cd code

/opt/ml/code


In [59]:
pwd

UsageError: CWD no longer exists - please use %cd to change directory.
